# Quick Simulation

In [ ]:
import time
import tkinter as tk
from ultralytics import YOLO
from PIL import Image, ImageTk, ImageDraw
import cv2

# Load the YOLO model
model = YOLO("best.pt")

# Create the main window
window = tk.Tk()
window.title("Traffic Light Simulation")

# Create canvas for two lights
canvas = tk.Canvas(window, width=800, height=400)
canvas.pack()

# Draw two traffic lights with initial red light
light1_red = canvas.create_oval(50, 20, 150, 120, fill="red")
light1_orange = canvas.create_oval(50, 125, 150, 175, fill="gray")
light1_green = canvas.create_oval(50, 180, 150, 280, fill="gray")

light2_red = canvas.create_oval(250, 20, 350, 120, fill="red")
light2_orange = canvas.create_oval(250, 125, 350, 175, fill="gray")
light2_green = canvas.create_oval(250, 180, 350, 280, fill="gray")

# Create labels to display vehicle counts and timers
label_info = tk.Label(window, text="", font=("Arial", 14))
label_info.pack()

# Create a frame to hold the images
frame_images = tk.Frame(window)
frame_images.pack()

# Helper function to set light colors
def set_light(light_red, light_orange, light_green, color, time=0):
    colors = {"red": ("red", "gray", "gray"), 
              "orange": ("gray", "orange", "gray"), 
              "green": ("gray", "gray", "green")}
    canvas.itemconfig(light_red, fill=colors[color][0])
    canvas.itemconfig(light_orange, fill=colors[color][1])
    canvas.itemconfig(light_green, fill=colors[color][2])
    window.update()

def draw_bounding_boxes(image_path, results):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    for box in results[0].boxes.data:
        x1, y1, x2, y2 = box[:4]  # Get the coordinates of the bounding box
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    
    return image

# Function to control traffic lights based on vehicle count
def update_lights(vehicles_road1, vehicles_road2, default_green_time=5, orange_time=1):
    # Update label info with vehicle counts
    label_info.config(text=f"Vehicles Road 1: {vehicles_road1}, Vehicles Road 2: {vehicles_road2}")
    
    # Set up variables to hold the traffic light timings for printing
    green_time_road1 = default_green_time
    green_time_road2 = default_green_time
    orange_time_road1 = orange_time
    orange_time_road2 = orange_time
    
    if abs(vehicles_road1 - vehicles_road2) <= 3:
        # Balanced traffic, default green light time
        print(f"Balanced traffic condition: green time = {default_green_time}")
        set_light(light1_red, light1_orange, light1_green, "green")
        time.sleep(default_green_time)
        
        set_light(light1_red, light1_orange, light1_green, "orange")
        time.sleep(orange_time)
        
        set_light(light1_red, light1_orange, light1_green, "red")
        set_light(light2_red, light2_orange, light2_green, "green")
        time.sleep(default_green_time)
        
        set_light(light2_red, light2_orange, light2_green, "orange")
        time.sleep(orange_time)
        
        set_light(light2_red, light2_orange, light2_green, "red")
        
    elif vehicles_road1 > vehicles_road2:
        # More traffic on road 1, extend green time
        additional_time = min(60, (vehicles_road1 - vehicles_road2) * 2)
        green_time_road1 += additional_time
        print(f"Road 1 has more traffic. Green time = {green_time_road1}")
        
        set_light(light1_red, light1_orange, light1_green, "green")
        time.sleep(green_time_road1)
        
        set_light(light1_red, light1_orange, light1_green, "orange")
        time.sleep(orange_time)
        
        set_light(light1_red, light1_orange, light1_green, "red")

        set_light(light2_red, light2_orange, light2_green, "green")
        time.sleep(default_green_time)
        
        set_light(light2_red, light2_orange, light2_green, "orange")
        time.sleep(orange_time)
        
        set_light(light2_red, light2_orange, light2_green, "red")
    else:
        # More traffic on road 2, extend green time
        additional_time = min(60, (vehicles_road2 - vehicles_road1) * 2)
        green_time_road2 += additional_time
        print(f"Road 2 has more traffic. Green time = {green_time_road2}")
        
        set_light(light2_red, light2_orange, light2_green, "green")
        time.sleep(green_time_road2)
        
        set_light(light2_red, light2_orange, light2_green, "orange")
        time.sleep(orange_time)
        
        set_light(light2_red, light2_orange, light2_green, "red")
        set_light(light1_red, light1_orange, light1_green, "green")
        time.sleep(default_green_time)
        
        set_light(light1_red, light1_orange, light1_green, "orange")
        time.sleep(orange_time)
        
        set_light(light1_red, light1_orange, light1_green, "red")
    
    return green_time_road1, green_time_road2, orange_time_road1, orange_time_road2


# Define the paths to the images for both roads
# images_road1 = [f"pic1\\image ({i}).jpg" for i in range(1, 9)]
# images_road2 = [f"pic1\\image ({i}).jpg" for i in range(9, 17)]

images_road1 = [f"pic2\\image{i}.jpeg" for i in range(1, 9)]
images_road2 = [f"pic2\\image{i}.jpeg" for i in range(9, 17)]

# Step 1: Print all class indices detected by YOLO for an image
sample_results = model(images_road1[0])  # Run on a sample image
for box in sample_results[0].boxes.data:
    print(f"Class index detected: {int(box[-1])}")

# Update vehicle_classes based on the printed indices for your model
vehicle_classes = [2, 3, 5, 7]  # Example class indices (e.g., 2: car, 3: truck, etc.)

# Step 2: Count all detected vehicle types based on the identified indices
for image_path_road1, image_path_road2 in zip(images_road1, images_road2):
    # Run YOLO prediction for both images
    results_road1 = model(image_path_road1)
    results_road2 = model(image_path_road2)
  
    # Count all detected vehicle classes in each road's image
    vehicles_road1 = sum(1 for box in results_road1[0].boxes.data if int(box[-1]) in vehicle_classes)
    vehicles_road2 = sum(1 for box in results_road2[0].boxes.data if int(box[-1]) in vehicle_classes)
    
    

    # Update traffic lights based on YOLO results and capture returned values
    green_time_road1, green_time_road2, orange_time_road1, orange_time_road2 = update_lights(vehicles_road1, vehicles_road2)

    image_with_boxes1 = draw_bounding_boxes(image_path_road1, results_road1)
    image_with_boxes2 = draw_bounding_boxes(image_path_road2, results_road2)
    
    # Print the green and orange times for both roads
    print(f"Road 1 green time: {green_time_road1}, Orange time: {orange_time_road1}")
    print(f"Road 2 green time: {green_time_road2}, Orange time: {orange_time_road2}")
    print(f"Total vehicles on road 1: {vehicles_road1}")
    print(f"Total vehicles on road 2: {vehicles_road2}")

    # Display images after prediction
    img1 = ImageTk.PhotoImage(image_with_boxes1.resize((400, 350)))
    img2 = ImageTk.PhotoImage(image_with_boxes2.resize((400, 350)))
    
    # Clear previous images
    for widget in frame_images.winfo_children():
        widget.destroy()
        
    # Display the current images
    label_road1 = tk.Label(frame_images, image=img1)
    label_road1.image = img1  # Keep a reference to avoid garbage collection
    label_road1.pack(side=tk.LEFT)

    label_road2 = tk.Label(frame_images, image=img2)
    label_road2.image = img2  # Keep a reference to avoid garbage collection
    label_road2.pack(side=tk.LEFT)

    # Log or save results as needed for each image
    print(f"Processed {image_path_road1} and {image_path_road2}")

    # Add a delay to simulate a real camera feed
    time.sleep(1)  # Delay of 1 second


# Run the Tkinter event loop
window.mainloop()


# Real Life Simulation

In [ ]:
import time
import tkinter as tk
from ultralytics import YOLO
from PIL import Image, ImageTk, ImageDraw
import cv2

# Load the YOLO model
model = YOLO("best.pt")

# Create the main window
window = tk.Tk()
window.title("Traffic Light Simulation")

# Create canvas for two lights
canvas = tk.Canvas(window, width=800, height=400)
canvas.pack()

# Draw two traffic lights with initial red light
light1_red = canvas.create_oval(50, 20, 150, 120, fill="red")
light1_orange = canvas.create_oval(50, 125, 150, 175, fill="gray")
light1_green = canvas.create_oval(50, 180, 150, 280, fill="gray")

light2_red = canvas.create_oval(250, 20, 350, 120, fill="red")
light2_orange = canvas.create_oval(250, 125, 350, 175, fill="gray")
light2_green = canvas.create_oval(250, 180, 350, 280, fill="gray")

# Create labels to display vehicle counts and timers
label_info = tk.Label(window, text="", font=("Arial", 14))
label_info.pack()

# Create a frame to hold the images
frame_images = tk.Frame(window)
frame_images.pack()

# Helper function to set light colors
def set_light(light_red, light_orange, light_green, color, time=0):
    colors = {"red": ("red", "gray", "gray"), 
              "orange": ("gray", "orange", "gray"), 
              "green": ("gray", "gray", "green")}
    canvas.itemconfig(light_red, fill=colors[color][0])
    canvas.itemconfig(light_orange, fill=colors[color][1])
    canvas.itemconfig(light_green, fill=colors[color][2])
    window.update()

def draw_bounding_boxes(image_path, results):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    for box in results[0].boxes.data:
        x1, y1, x2, y2 = box[:4]  # Get the coordinates of the bounding box
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    
    return image

# Function to control traffic lights based on vehicle count
def update_lights(vehicles_road1, vehicles_road2, default_green_time=25, orange_time=5):
    # Update label info with vehicle counts
    label_info.config(text=f"Vehicles Road 1: {vehicles_road1}, Vehicles Road 2: {vehicles_road2}")
    
    # Set up variables to hold the traffic light timings for printing
    green_time_road1 = default_green_time
    green_time_road2 = default_green_time
    orange_time_road1 = orange_time
    orange_time_road2 = orange_time
    
    if abs(vehicles_road1 - vehicles_road2) <= 3:
        # Balanced traffic, default green light time
        print(f"Balanced traffic condition: green time = {default_green_time}")
        set_light(light1_red, light1_orange, light1_green, "green")
        time.sleep(default_green_time)
        
        set_light(light1_red, light1_orange, light1_green, "orange")
        time.sleep(orange_time)
        
        set_light(light1_red, light1_orange, light1_green, "red")
        set_light(light2_red, light2_orange, light2_green, "green")
        time.sleep(default_green_time)
        
        set_light(light2_red, light2_orange, light2_green, "orange")
        time.sleep(orange_time)
        
        set_light(light2_red, light2_orange, light2_green, "red")
        
    elif vehicles_road1 > vehicles_road2:
        # More traffic on road 1, extend green time
        additional_time = min(60, (vehicles_road1 - vehicles_road2) * 2)
        green_time_road1 += additional_time
        print(f"Road 1 has more traffic. Green time = {green_time_road1}")
        
        set_light(light1_red, light1_orange, light1_green, "green")
        time.sleep(green_time_road1)
        
        set_light(light1_red, light1_orange, light1_green, "orange")
        time.sleep(orange_time)
        
        set_light(light1_red, light1_orange, light1_green, "red")
        set_light(light2_red, light2_orange, light2_green, "green")
        time.sleep(default_green_time)
        
        set_light(light2_red, light2_orange, light2_green, "orange")
        time.sleep(orange_time)
        
        set_light(light2_red, light2_orange, light2_green, "red")
    else:
        # More traffic on road 2, extend green time
        additional_time = min(60, (vehicles_road2 - vehicles_road1) * 2)
        green_time_road2 += additional_time
        print(f"Road 2 has more traffic. Green time = {green_time_road2}")
        
        set_light(light2_red, light2_orange, light2_green, "green")
        time.sleep(green_time_road2)
        
        set_light(light2_red, light2_orange, light2_green, "orange")
        time.sleep(orange_time)
        
        set_light(light2_red, light2_orange, light2_green, "red")
        set_light(light1_red, light1_orange, light1_green, "green")
        time.sleep(default_green_time)
        
        set_light(light1_red, light1_orange, light1_green, "orange")
        time.sleep(orange_time)
        
        set_light(light1_red, light1_orange, light1_green, "red")
    
    return green_time_road1, green_time_road2, orange_time_road1, orange_time_road2


# Define the paths to the images for both roads
# images_road1 = [f"pic1\\image ({i}).jpg" for i in range(1, 9)]
# images_road2 = [f"pic1\\image ({i}).jpg" for i in range(9, 17)]

images_road1 = [f"pic2\\image{i}.jpeg" for i in range(1, 9)]
images_road2 = [f"pic2\\image{i}.jpeg" for i in range(9, 17)]

# Step 1: Print all class indices detected by YOLO for an image
sample_results = model(images_road1[0])  # Run on a sample image
for box in sample_results[0].boxes.data:
    print(f"Class index detected: {int(box[-1])}")

# Update vehicle_classes based on the printed indices for your model
vehicle_classes = [2, 3, 5, 7]  # Example class indices (e.g., 2: car, 3: truck, etc.)

# Step 2: Count all detected vehicle types based on the identified indices
for image_path_road1, image_path_road2 in zip(images_road1, images_road2):
    # Run YOLO prediction for both images
    results_road1 = model(image_path_road1)
    results_road2 = model(image_path_road2)
  
    # Count all detected vehicle classes in each road's image
    vehicles_road1 = sum(1 for box in results_road1[0].boxes.data if int(box[-1]) in vehicle_classes)
    vehicles_road2 = sum(1 for box in results_road2[0].boxes.data if int(box[-1]) in vehicle_classes)
    
    

    # Update traffic lights based on YOLO results and capture returned values
    green_time_road1, green_time_road2, orange_time_road1, orange_time_road2 = update_lights(vehicles_road1, vehicles_road2)

    image_with_boxes1 = draw_bounding_boxes(image_path_road1, results_road1)
    image_with_boxes2 = draw_bounding_boxes(image_path_road2, results_road2)
    
    # Print the green and orange times for both roads
    print(f"Road 1 green time: {green_time_road1}, Orange time: {orange_time_road1}")
    print(f"Road 2 green time: {green_time_road2}, Orange time: {orange_time_road2}")
    print(f"Total vehicles on road 1: {vehicles_road1}")
    print(f"Total vehicles on road 2: {vehicles_road2}")

    # Display images after prediction
    img1 = ImageTk.PhotoImage(image_with_boxes1.resize((400, 350)))
    img2 = ImageTk.PhotoImage(image_with_boxes2.resize((400, 350)))
    
    # Clear previous images
    for widget in frame_images.winfo_children():
        widget.destroy()
        
    # Display the current images
    label_road1 = tk.Label(frame_images, image=img1)
    label_road1.image = img1  # Keep a reference to avoid garbage collection
    label_road1.pack(side=tk.LEFT)

    label_road2 = tk.Label(frame_images, image=img2)
    label_road2.image = img2  # Keep a reference to avoid garbage collection
    label_road2.pack(side=tk.LEFT)

    # Log or save results as needed for each image
    print(f"Processed {image_path_road1} and {image_path_road2}")

    # Add a delay to simulate a real camera feed
    time.sleep(1)  # Delay of 1 second


# Run the Tkinter event loop
window.mainloop()


In [ ]:
import time
from ultralytics import YOLO
import requests

# Adresse IP de l'ESP8266
ESP8266_IP = "http://192.168.xxx.xxx"  # Remplace par l'adresse IP de l'ESP8266
PORT = 80


# Load the YOLO model
model = YOLO("best.pt")

def send_traffic_data(vehicles_road1, vehicles_road2):
    url = f"{ESP8266_IP}:{PORT}/update?road1={vehicles_road1}&road2={vehicles_road2}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print("Data sent successfully!")
        else:
            print("Failed to send data. Status code:", response.status_code)
    except Exception as e:
        print("Error:", e)

images_road1 = [f"pic2\\image{i}.jpeg" for i in range(1, 9)]
images_road2 = [f"pic2\\image{i}.jpeg" for i in range(9, 17)]

# Step 1: Print all class indices detected by YOLO for an image
sample_results = model(images_road1[0])  # Run on a sample image
for box in sample_results[0].boxes.data:
    print(f"Class index detected: {int(box[-1])}")

# Update vehicle_classes based on the printed indices for your model
vehicle_classes = [2, 3, 5, 7]  # Example class indices (e.g., 2: car, 3: truck, etc.)

# Step 2: Count all detected vehicle types based on the identified indices
for image_path_road1, image_path_road2 in zip(images_road1, images_road2):
    # Run YOLO prediction for both images
    results_road1 = model(image_path_road1)
    results_road2 = model(image_path_road2)
  
    # Count all detected vehicle classes in each road's image
    vehicles_road1 = sum(1 for box in results_road1[0].boxes.data if int(box[-1]) in vehicle_classes)
    vehicles_road2 = sum(1 for box in results_road2[0].boxes.data if int(box[-1]) in vehicle_classes)
    send_traffic_data(vehicles_road1, vehicles_road2)

    time.sleep(1)  # Delay of 1 second

